In [ ]:
# 1
library(tidyr)
library(readr)
library(ggplot2)
library(dplyr)
library(RColorBrewer)
library(viridis)
library(ggplot2)
library(colorspace)
library(pheatmap)
library(reshape2)  
library(ComplexHeatmap)
library(circlize)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: viridisLite



In [ ]:
mutation_clinical=read.table("256sample_mutation_paired_clinical_filter.csv",sep=",",quote="\"",header=T)

In [14]:
mutation_sample <- mutation_clinical %>%
  group_by(Sample, Gene.MT_ensGene) %>%
  mutate(Gene_count = if (first(Gene.MT_ensGene) != "missing") n() else 0) %>%
  ungroup()

In [ ]:
mutation_gene_count <- unique(mutation_sample %>%
  select(ID,Sample, class, Gene.MT_ensGene, Gene_count,gender,age,T.stage,N.stage,BRAF.V600E))

In [ ]:
mutation_wide <- dcast(mutation_gene_count, ID+Sample +gender+age+T.stage+N.stage+BRAF.V600E + class ~ Gene.MT_ensGene,
                       value.var = 'Gene_count')

In [22]:
mutation_wide[is.na(mutation_wide)] <- 0

In [26]:
mutation_wide <- mutation_wide[, names(mutation_wide) != "missing"]

In [32]:
gene_order <- mutation_gene_count %>%
  filter(class == "adjacent", Gene.MT_ensGene != "missing") %>%
  group_by(Gene.MT_ensGene) %>%
  summarise(total_count = sum(Gene_count, na.rm = TRUE)) %>%
  arrange(desc(total_count)) %>%
  pull(Gene.MT_ensGene)

In [ ]:
mutation_wide <- mutation_wide %>%
  mutate(
    class = factor(class, levels = c("adjacent", "tumor")),
    gender = factor(gender, levels = c("male", "female")),
    ID = as.character(ID)  
  )


adjacent_data <- mutation_wide %>%
  filter(class == "adjacent") %>%
  arrange(gender, age)


ordered_IDs <- adjacent_data$ID


tumor_data <- mutation_wide %>%
  filter(class == "tumor") %>%
  mutate(ID = factor(ID, levels = ordered_IDs)) %>%  
  arrange(ID, gender, age) %>%
  mutate(ID = as.character(ID)) 

heatmap_data <- bind_rows(adjacent_data, tumor_data) %>%
  select(ID, Sample, class, gender, age, T.stage, N.stage, BRAF.V600E, all_of(gene_order))


In [53]:
clinical_data=t(heatmap_data[1:8,])

In [ ]:
colnames(heatmap_data) <- as.character(heatmap_data[2, ])
heatmap_data_new<- as.matrix(heatmap_data[-c(1:8), ])
heatmap_matrix_new <- apply(heatmap_data_new, 2, as.numeric)

WES-K100531,WES-K100571,KY-WES-137019,KY-131772,WES-K100616,KY-131806,WES-K100569,KY-131732,KY-WES-136997,WES-K100593,⋯,WES-K84477,KY-WES-137007,KY-131843,KY-WES-137023,WES-K100560,KY-WES-136999,WES-K101747,KY-131783,KY-WES-136973,KY-WES-137002
0,0,0,0,0,10,2,10,0,2,⋯,0,0,0,0,4,5,28,0,22,0
0,0,0,0,0,10,2,10,0,2,⋯,0,0,0,0,4,4,28,0,22,0
0,0,0,0,0,10,2,10,0,2,⋯,0,0,0,0,4,4,28,0,22,0
1,2,1,0,0,0,4,4,0,0,⋯,0,0,0,0,3,0,5,1,1,1
1,2,1,0,0,0,4,4,0,0,⋯,0,0,0,0,3,0,5,1,1,1
0,0,0,0,0,0,0,0,4,0,⋯,0,0,1,0,0,0,4,0,1,0


In [73]:
heatmap_matrix_scaled <- t(scale(t(heatmap_matrix_new)))

In [ ]:
annotation_col <- data.frame(
  Class = factor(clinical_data[, 3]),
  Gender = factor(clinical_data[, 4]),
  Age = as.numeric(clinical_data[, 5]),  
  T_stage = factor(clinical_data[, 6]),
  N_stage = factor(clinical_data[, 7]),
  BRAF_V600E = factor(clinical_data[, 8])  
)


rownames(annotation_col) <- clinical_data[, 2]


Class_colors <- c("adjacent" = "#0072B2", "tumor" = "#D55E00")
Gender_colors <- c("male" = "#A4CEE4", "female" = "#DDA47B")
T_stage_colors <- c("T1" = "#cfa5b3", "T2" = "#8BBEBF", "T3" = "#aba94d", "T4" = "#F08650")
N_stage_colors <- c("N0" = "#99D9EA", "N1" = "#7092BE")
BRAF_colors <- c("0" = "#8AADBE", "1" = "#dcc7d0")


ha = HeatmapAnnotation(
  Class = annotation_col$Class,
  Gender = annotation_col$Gender,
  Age = annotation_col$Age,
  T_stage = annotation_col$T_stage,
  N_stage = annotation_col$N_stage,
  BRAF_V600E = annotation_col$BRAF_V600E,
  col = list(
    Class = Class_colors,
    Gender = Gender_colors,
    Age = colorRamp2(c(min(annotation_col$Age), max(annotation_col$Age)), c("#496fb0", "#9F4E59")),
    T_stage = T_stage_colors,
    N_stage = N_stage_colors,
    BRAF_V600E = BRAF_colors
  )
)


heatmap_matrix_scaled <- t(scale(t(heatmap_matrix_new)))
min_val=min(heatmap_matrix_scaled)
max_val=max(heatmap_matrix_scaled)

col_fun <- colorRamp2(c(-16, 0, 16), c("blue", "white", "red"))


pdf("Thyroid cancer gene_mutation_counts_adjacent.pdf", width = 15, height = 8)
p=Heatmap(
  heatmap_matrix_scaled,
  name = "Value",
  col = col_fun,
  top_annotation = ha,
  show_column_names = FALSE,
  show_row_names = TRUE,
  cluster_columns = FALSE,
  cluster_rows = FALSE,
  row_names_gp = gpar(fontsize = 15)
)
draw(p)
dev.off()

pdf 
  2

In [ ]:
# 2